# cookie
- 在浏览器
- session: 在服务器
- set_cookie(key, value，seconds)
- del_cookie()
- 令牌有过期时间，可以设置
- 服务端有令牌时间，mysql,mongodb

- 下面两段代码实现了设置令牌，和撤销令牌

In [ ]:
from django.contrib.auth.hashers import make_password, check_password
from django.http import HttpResponseRedirect, HttpResponse
from django.shortcuts import render

# Create your views here.
from uauth.models import Users


def regist(request):

    if request.method == 'GET':
        return render(request, 'register.html')

    if request.method == 'POST':
        # 注册
        name = request.POST.get('name')
        password = request.POST.get('password')
        # 加密
        password = make_password(password)

        Users.objects.create(
            u_name=name,
            u_password=password
        )
        return HttpResponseRedirect('/uauth/login/')


def login(request):

    if request.method == 'GET':
        return render(request, 'login.html')

    if request.method == 'POST':
        # 如果登录成功，绑定参数到cookie中，set_cookie
        name = request.POST.get('name')
        password = request.POST.get('password')

        if Users.objects.filter(u_name=name).exists():
            user = Users.objects.get(u_name=name)
            if check_password(password, user.u_password):
                ticket = 'jkdjasfkn'
                # 绑定令牌到cookie里面
                response = HttpResponse()
                response.set_cookie('ticket', ticket)
                # 存在服务端
                user.u_ticket = ticket
                user.save()
                return response
        else:
            return HttpResponse('用户不存在')


def logout(request):

    if request.method == 'GET':
        response = HttpResponse()
        # 撤销令牌
        response.delete_cookie('ticket')
        return HttpResponseRedirect('/uauth/login/')

In [ ]:
def index(request):

    if request.method == 'GET':
        # 获取所有学生信息
        # 验证令牌
        ticket = request.COOKIES.get('ticket')
        if not ticket:
            return HttpResponseRedirect('/uauth/login/')
        if Users.objects.filter(u_ticket=ticket).exists():
            stuinfos = StudentInfo.objects.all()
            return render(request, 'index.html', {'stuinfos':stuinfos})
        else:
            return HttpResponseRedirect('/uauth/login/')


def addStu(request):

    if request.method == 'GET':

        return render(request, 'addStu.html')

    if request.method == 'POST':
        # 跳转学生详情页面
        name = request.POST.get('name')
        tel = request.POST.get('tel')

        stu = Student.objects.create(s_name=name, s_tel=tel)

        return HttpResponseRedirect(reverse('stu:addStuInfo', kwargs={'stu_id': stu.id}))


def addStuInfo(request, stu_id):

    if request.method == 'GET':
        return render(request, 'addStuInfo.html', {'stu_id': stu_id})
    if request.method == 'POST':

        # stu_id = request.POST.get('stu_id')
        addr = request.POST.get('addr')

        StudentInfo.objects.create(
            i_addr=addr,
            s_id=stu_id
        )

        return HttpResponseRedirect('/stu/index/')

In [ ]:
"""
作业：
1）使用django自带的登录注册
"""